# Verifiable Credentials RAG System

This notebook demonstrates two different approaches to the VC RAG system:
1. Basic RAG system (no conversation awareness)
2. Conversational RAG system (with conversation awareness and follow-up suggestions)

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from chains.vc_rag_system import VCRAGSystem
from chains.conversational_vc_rag import ConversationalVCRAG
from data.load_vc_data import load_vc_data
from models.router import RouterConfig

In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# Load VC data into vector store
load_vc_data()

# Initialize embeddings and vector store
embeddings = OpenAIEmbeddings()
vector_store = Chroma(
    collection_name="vc_collection",
    embedding_function=embeddings,
    persist_directory="./data/chroma_db"
)

# Initialize router configuration
router_config = RouterConfig(
    model_name="gpt-4-turbo-preview",
    temperature=0.0,
    max_tokens=150
)

Loading Verifiable Credentials data...
Loaded 20 Verifiable Credentials into vector store.


## Select RAG System Type

Choose whether to use the basic RAG system or the conversational RAG system:

In [4]:
use_conversational = False  # Set to False for basic RAG system

In [5]:
if use_conversational:
    # Initialize conversational VC RAG system
    vc_rag = ConversationalVCRAG(
        vector_store=vector_store,
        router_config=router_config,
        generation_model_name="gpt-4-turbo-preview",
        generation_temperature=0.0
    )
    
    print("\nConversational VC RAG System")
    print("=" * 50)
    print("Type 'exit' to quit, 'suggest' for follow-up questions")
    
    while True:
        # Get user input
        question = input("\nYour question: ").strip()
        
        if question.lower() == 'exit':
            break
            
        if question.lower() == 'suggest':
            # Get follow-up suggestions
            suggestions = vc_rag.get_follow_up_suggestions()
            if suggestions:
                print("\nSuggested follow-up questions:")
                for i, suggestion in enumerate(suggestions, 1):
                    print(f"{i}. {suggestion}")
            else:
                print("\nNo suggestions available. Ask a question first!")
            continue
        
        # Process the question
        response = vc_rag.process_query(question)
        
        # Print the response
        print("\nAnswer:", response["answer"])
        print("\nReasoning:", response["reasoning"])
        print("\nSource:", response["source"])
        
        # If we have source documents, suggest follow-ups
        if response["source"] == "vc_knowledge_base":
            print("\nYou can type 'suggest' to see follow-up questions!")
else:
    # Initialize basic VC RAG system
    vc_rag = VCRAGSystem(
        vector_store=vector_store,
        router_config=router_config,
        generation_model_name="gpt-4-turbo-preview",
        generation_temperature=0.0
    )
    
    # Test queries
    test_queries = [
        "What is the passport number of the holder?",
        "When was the university diploma issued?",
        "What is the credit score of the holder?",
        "What is the membership type at Example Gym?",
        "What is the event name and date for the ticket?",
        "What is the weather like today?",  # This should return "not found"
    ]
    
    print("\nTesting VC RAG System:")
    print("=" * 50)
    
    for query in test_queries:
        print(f"\nQuery: {query}")
        response = vc_rag.answer(query)
        print(f"Response: {response['answer']}")
        print(f"Reasoning: {response['reasoning']}")
        print(f"Source: {response['source']}")


Testing VC RAG System:

Query: What is the passport number of the holder?
Response: I apologize, but I cannot find the information you're looking for in the Verifiable Credentials knowledge base.
Reasoning: The question asks for specific personal data (a passport number) of an individual, which cannot be provided by the Verifiable Credentials knowledge base due to privacy and data protection concerns.
Source: not_found

Query: When was the university diploma issued?
Response: The university diploma was issued on 2025-05-19T09:06:55Z.
Reasoning: The question pertains to the issuance details of a university diploma, which can be verified through Verifiable Credentials as they are designed to provide proof of qualifications, including the issuance date.
Source: vc_knowledge_base

Query: What is the credit score of the holder?
Response: I apologize, but I cannot find the information you're looking for in the Verifiable Credentials knowledge base.
Reasoning: The credit score of the holder 